In [ ]:
pip install dash

In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]
    
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf
prov=get_province_shape()

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

# My dashboard

Populate the dropdown

In [ ]:
#NEW functions
def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list


This first part is just for the layout of the elements

In [ ]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Bugs_project: Air quality analysis'),

    html.Div([
    html.H2('DV_5 Single Sensor Time Series'),
    html.P('Start by selecting a pollutant, then select one among all the sensors that measures that pollutant, and then visualize in the graph the time series of the monthly average of the data measured by that sensor'),

    #Dropdown to select the pollutant
        dcc.Dropdown(
        id='pollutant-dropdown',
        options=pollutant_dropdown(),
        placeholder="Select a pollutant",
    ),

    #Dropdown to select the sensor
        dcc.Dropdown(
        id='sensor-dropdown',
        placeholder="Select a sensor",
    ),
    
    #Visualiza the graph
    dcc.Graph(id='time_series_single_sensor'),
    ], style={'padding': 20, 'border': '1px solid #ccc'}),

    html.Div([
    html.H2('DV_6 Location of all the sensors of a selected pollutant'),
    html.P('Chose a pollutant, and then visualize on a map the location af all the sensors of that pollutant. The sensor will appear red if it\'s still active, and will appear gray if it is not active anymore. Clicking on the icon of a sensor the user can visualiza the id of the sensor, the name of the station in which the sensor is located and date in which the sensor was deactivated, if it is not active anymore'),
    
    #Dropdown to select the pollutant
        dcc.Dropdown(
        id='all-pollutant-dropdown',
        options=all_pollutant_dropdown(),
        placeholder="Select a pollutant",
    ),

    ], style={'padding': 20, 'border': '1px solid #ccc'}),
])

#Callback to get the sensor list
@app.callback(
    Output('sensor-dropdown', 'options'),
    Input('pollutant-dropdown','value')
)
def sensor_dropdown(selected_pollutant):
    if not selected_pollutant:
        options = []
        return options
    t=requests.post(url="http://127.0.0.1:5000/api/sensor", json={"var_pollutant": selected_pollutant})
    list_sensors = read_response(t)
    options = [{'label': f'{sensor['nome_stazione']} - {sensor['id_sensore']}', 'value': sensor['id_sensore']} for sensor in list_sensors]
    return options

#Callback to print the first graph
@app.callback(
    Output('time_series_single_sensor','figure'),
    [Input('pollutant-dropdown','value'),
     Input('sensor-dropdown','value')]
)
def time_series(selected_pollutant,sensor_id):
    data = {"var_sensor_id": sensor_id, "var_pollutant": selected_pollutant} #Right now data is a dictionary
    # send the request to the server, we use post because we are sending data to the server
    t=requests.post(url="http://127.0.0.1:5000/api/DV_5", json=data) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    df = pd.DataFrame(data)
    df['month'] = pd.to_datetime(df['month'], errors='coerce') #keeps just year and month for the date
    df['month'] = df['month'].dt.strftime('%b %Y')
    unit=get_measurement_unit(selected_pollutant) #gets the unit of measurement for a pollutant
    line_chart = px.line(df, x='month', y='monthly_avg', title=f'Monthly average for {selected_pollutant} of sensor {sensor_id} in {unit}')
    return line_chart

# Run the app in Jupyter notebook
print("Starting Dashboard...")
#app.run_server(port=8089)
app.run(port=8089, debug=True)

# Codice suo

In [ ]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Bugs_project: Air quality analysis'),

    # Form to add a new sale
    html.Div([
        html.H3('Add New Sale'),
        html.Label('Date'),
        dcc.Input(id='input-date', type='date', value='2024-01-01'),
        
        html.Label('Product'),
        dcc.Input(id='input-product', type='text', value='Product A'),
        
        html.Label('Quantity'),
        dcc.Input(id='input-quantity', type='number', value=1),
        
        html.Label('Price'),
        dcc.Input(id='input-price', type='number', value=99.99),
        
        html.Button('Add Sale', id='add-sale-button', n_clicks=0)
    ], style={'padding': 20, 'border': '1px solid #ccc'}),

    # Dropdown and charts
    dcc.Dropdown(
        id='product-dropdown',
        value='Product A',
    ),
    dcc.Graph(id='sales-bar-chart'),
    dcc.Graph(id='sales-line-chart')
])

# Define the callback to add a new sale
@app.callback(
    Output('product-dropdown', 'value'),
    Input('add-sale-button', 'n_clicks'),
    State('input-date', 'value'),
    State('input-product', 'value'),
    State('input-quantity', 'value'),
    State('input-price', 'value')
)
def add_sale(n_clicks, date, product, quantity, price):
    if n_clicks > 0:
        new_sale = {
            'date': date,
            'product': product,
            'quantity': quantity,
            'price': price
        }
        requests.post('http://127.0.0.1:5000/sales', json=new_sale)
        return product
    return 'Product A'

# Define the callback to update charts
@app.callback(
    [Output('sales-bar-chart', 'figure'),
    Output('sales-line-chart', 'figure'),     
    Output('product-dropdown', 'options')],
    [Input('product-dropdown', 'value')]
)
def update_charts(selected_product):
    print(selected_product)
    df = fetch_data()
    options = [{'label': product, 'value': product} for product in np.sort(df['product'].unique())]

    if selected_product not in df['product'].unique():
        selected_product = np.sort(df['product'].unique())[0]
    
    filtered_df = df[df['product'] == selected_product]

    bar_chart = px.bar(filtered_df, x='date', y='quantity', title=f'Sales Quantity for {selected_product}')
    line_chart = px.line(filtered_df, x='date', y='price', title=f'Sales Price over Time for {selected_product}')

    return bar_chart, line_chart, options


# Run the app in Jupyter notebook
print("Starting Dashboard...")
app.run_server(port=8089)
